In [2]:
import argparse
import json
import logging
import os
import random
from io import open
import numpy as np

from tensorboardX import SummaryWriter
from tqdm import tqdm
from bisect import bisect
import yaml
from easydict import EasyDict as edict
import sys
import pdb

import torch
import torch.nn.functional as F
import torch.nn as nn

from vilbert.task_utils import (
    LoadDatasetEval,
    LoadLosses,
    ForwardModelsTrain,
    ForwardModelsVal,
    EvaluatingModel,
)

import vilbert.utils as utils
import torch.distributed as dist

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [4]:
parser = argparse.ArgumentParser()

parser.add_argument(
    "--bert_model",
    default="bert-base-uncased",
    type=str,
    help="Bert pre-trained model selected in the list: bert-base-uncased, "
    "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.",
)
parser.add_argument(
    "--from_pretrained",
    default="bert-base-uncased",
    type=str,
    help="Bert pre-trained model selected in the list: bert-base-uncased, "
    "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.",
)
parser.add_argument(
    "--output_dir",
    default="results",
    type=str,
    help="The output directory where the model checkpoints will be written.",
)
parser.add_argument(
    "--config_file",
    default="config/bert_config.json",
    type=str,
    help="The config file which specified the model details.",
)
parser.add_argument(
    "--no_cuda", action="store_true", help="Whether not to use CUDA when available"
)
parser.add_argument(
    "--do_lower_case",
    default=True,
    type=bool,
    help="Whether to lower case the input text. True for uncased models, False for cased models.",
)
parser.add_argument(
    "--local_rank",
    type=int,
    default=-1,
    help="local_rank for distributed training on gpus",
)
parser.add_argument(
    "--seed", type=int, default=42, help="random seed for initialization"
)
parser.add_argument(
    "--fp16",
    action="store_true",
    help="Whether to use 16-bit float precision instead of 32-bit",
)
parser.add_argument(
    "--loss_scale",
    type=float,
    default=0,
    help="Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True.\n"
    "0 (default value): dynamic loss scaling.\n"
    "Positive power of 2: static loss scaling value.\n",
)
parser.add_argument(
    "--num_workers",
    type=int,
    default=16,
    help="Number of workers in the dataloader.",
)
parser.add_argument(
    "--save_name", default="", type=str, help="save name for training."
)
parser.add_argument(
    "--use_chunk",
    default=0,
    type=float,
    help="whether use chunck for parallel training.",
)
parser.add_argument(
    "--batch_size", default=30, type=int, help="what is the batch size?"
)
parser.add_argument(
    "--tasks", default="", type=str, help="1-2-3... training task separate by -"
)
parser.add_argument(
    "--in_memory",
    default=False,
    type=bool,
    help="whether use chunck for parallel training.",
)
parser.add_argument(
    "--baseline", action="store_true", help="whether use single stream baseline."
)
parser.add_argument("--split", default="", type=str, help="which split to use.")
parser.add_argument(
    "--dynamic_attention",
    action="store_true",
    help="whether use dynamic attention.",
)
parser.add_argument(
    "--clean_train_sets",
    default=True,
    type=bool,
    help="whether clean train sets for multitask data.",
)
parser.add_argument(
    "--visual_target",
    default=0,
    type=int,
    help="which target to use for visual branch. \
    0: soft label, \
    1: regress the feature, \
    2: NCE loss.",
)
parser.add_argument(
    "--task_specific_tokens",
    action="store_true",
    help="whether to use task specific tokens for the multi-task learning.",
)

#args = parser.parse_args()
args = parser.parse_args(['--bert_model', 'bert-base-uncased',
                          '--from_pretrained', 'save/VQA_bert_base_6layer_6conect-finetune_from_multi_task_model/pytorch_model_19.bin',
                          '--config_file', 'config/bert_base_6layer_6conect.json',
                          '--tasks', '1',
                          '--split', 'minval',
                          '--save_name', 'testing',
                          '--task_specific_tokens'])


with open("vilbert_tasks.yml", "r") as f:
    task_cfg = edict(yaml.safe_load(f))

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

if args.baseline:
    from pytorch_transformers.modeling_bert import BertConfig
    from vilbert.basebert import BaseBertForVLTasks
else:
    from vilbert.vilbert import BertConfig
    from vilbert.vilbert import VILBertForVLTasks

task_names = []
for i, task_id in enumerate(args.tasks.split("-")):
    task = "TASK" + task_id
    name = task_cfg[task]["name"]
    task_names.append(name)

# timeStamp = '-'.join(task_names) + '_' + args.config_file.split('/')[1].split('.')[0]
timeStamp = args.from_pretrained.split("/")[-1] + "-" + args.save_name
savePath = os.path.join(args.output_dir, timeStamp)
config = BertConfig.from_json_file(args.config_file)

if args.task_specific_tokens:
    config.task_specific_tokens = True

if args.local_rank == -1 or args.no_cuda:
    device = torch.device(
        "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    )
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend="nccl")

logger.info(
    "device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16
    )
)

default_gpu = False
if dist.is_available() and args.local_rank != -1:
    rank = dist.get_rank()
    if rank == 0:
        default_gpu = True
else:
    default_gpu = True

if default_gpu and not os.path.exists(savePath):
    os.makedirs(savePath)

task_batch_size, task_num_iters, task_ids, task_datasets_val, task_dataloader_val = LoadDatasetEval(
    args, task_cfg, args.tasks.split("-")
)

tbLogger = utils.tbLogger(
    timeStamp,
    savePath,
    task_names,
    task_ids,
    task_num_iters,
    1,
    save_logger=False,
    txt_name="eval.txt",
)
num_labels = max([dataset.num_labels for dataset in task_datasets_val.values()])

if args.dynamic_attention:
    config.dynamic_attention = True
if "roberta" in args.bert_model:
    config.model = "roberta"

if args.visual_target == 0:
    config.v_target_size = 1601
    config.visual_target = args.visual_target
else:
    config.v_target_size = 2048
    config.visual_target = args.visual_target

if args.task_specific_tokens:
    config.task_specific_tokens = True

if args.baseline:
    model = BaseBertForVLTasks.from_pretrained(
        args.from_pretrained,
        config=config,
        num_labels=num_labels,
        default_gpu=default_gpu,
    )
else:
    model = VILBertForVLTasks.from_pretrained(
        args.from_pretrained,
        config=config,
        num_labels=num_labels,
        default_gpu=default_gpu,
    )

task_losses = LoadLosses(args, task_cfg, args.tasks.split("-"))
model.to(device)
if args.local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError(
            "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training."
        )
    model = DDP(model, delay_allreduce=True)

elif n_gpu > 1:
    model = nn.DataParallel(model)

print("***** Running evaluation *****")
print("  Num Iters: ", task_num_iters)
print("  Batch size: ", task_batch_size)

model.eval()
# when run evaluate, we run each task sequentially.
for task_id in task_ids:
    results = []
    others = []
    for i, batch in enumerate(task_dataloader_val[task_id]):
        loss, score, batch_size, results, others = EvaluatingModel(
            args,
            task_cfg,
            device,
            task_id,
            batch,
            model,
            task_dataloader_val,
            task_losses,
            results,
            others,
        )

        tbLogger.step_val(0, float(loss), float(score), task_id, batch_size, "val")

        sys.stdout.write("%d/%d\r" % (i, len(task_dataloader_val[task_id])))
        sys.stdout.flush()
    # save the result or evaluate the result.
    ave_score = tbLogger.showLossVal(task_id)

    if args.split:
        json_path = os.path.join(savePath, args.split)
    else:
        json_path = os.path.join(savePath, task_cfg[task_id]["val_split"])

    json.dump(results, open(json_path + "_result.json", "w"))
    json.dump(others, open(json_path + "_others.json", "w"))

07/28/2020 15:50:52 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
07/28/2020 15:50:52 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/aloui/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/28/2020 15:50:52 - INFO - vilbert.task_utils -   Loading VQA Dataset with batch size 30
07/28/2020 15:50:52 - INFO - vilbert.datasets.vqa_dataset -   Loading from datasets/VQA/cache/VQA_minval_23_cleaned.pkl
07/28/2020 15:50:53 - INFO - vilbert.utils -   logging file at: pytorch_model_19.bin-testing
07/28/2020 15:50:53 - INFO - vilbert.utils -   loading weights file save/VQA_bert_base_6layer_6conect-finetune_from_multi_task_model/pytorch_model_19.bin


***** Running evaluation *****
  Num Iters:  {'TASK1': 100}
  Batch size:  {'TASK1': 30}


07/28/2020 15:51:12 - INFO - vilbert.utils -   Eval task TASK1 on iteration 0 
07/28/2020 15:51:12 - INFO - vilbert.utils -   Validation [VQA]: loss 0.000 score 0.000 


In [5]:
%whos

Variable              Type                          Data/Info
-------------------------------------------------------------
BertConfig            type                          <class 'vilbert.vilbert.BertConfig'>
EvaluatingModel       function                      <function EvaluatingModel at 0x7fe0f3700b70>
F                     module                        <module 'torch.nn.functio<...>/torch/nn/functional.py'>
ForwardModelsTrain    function                      <function ForwardModelsTrain at 0x7fe0f37008c8>
ForwardModelsVal      function                      <function ForwardModelsVal at 0x7fe0f3700840>
LoadDatasetEval       function                      <function LoadDatasetEval at 0x7fe0f3700a60>
LoadLosses            function                      <function LoadLosses at 0x7fe0f3700950>
SummaryWriter         type                          <class 'tensorboardX.writer.SummaryWriter'>
VILBertForVLTasks     type                          <class 'vilbert.vilbert.VILBertForVLTasks'>
arg

In [6]:
args

Namespace(baseline=False, batch_size=30, bert_model='bert-base-uncased', clean_train_sets=True, config_file='config/bert_base_6layer_6conect.json', do_lower_case=True, dynamic_attention=False, fp16=False, from_pretrained='save/VQA_bert_base_6layer_6conect-finetune_from_multi_task_model/pytorch_model_19.bin', in_memory=False, local_rank=-1, loss_scale=0, no_cuda=False, num_workers=16, output_dir='results', save_name='testing', seed=42, split='minval', task_specific_tokens=True, tasks='1', use_chunk=0, visual_target=0)

In [7]:
batch = tuple(t.cuda(device=device, non_blocking=True) for t in batch)

features, spatials, image_mask, question, target, input_mask, segment_ids, co_attention_mask, question_id = (
    batch
)

task_tokens = question.new().resize_(question.size(0), 1).fill_(int(task_id[4:]))

In [8]:
target.shape

torch.Size([30, 3129])

In [9]:
vil_prediction, vil_prediction_gqa, vil_logit, vil_binary_prediction, vil_tri_prediction, vision_prediction, vision_logit, linguisic_prediction, linguisic_logit, _ = model(
    question, 
    features, 
    spatials, 
    segment_ids, 
    input_mask, 
    image_mask, 
    co_attention_mask, 
    task_tokens,)

In [10]:
logits = torch.max(vil_prediction, 1)[1].data  # argmax
one_hots = torch.zeros(*target.size()).cuda()
one_hots.scatter_(1, logits.view(-1, 1), 1)
scores = one_hots * target

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [18]:
scores.sum() / features.size(0)

tensor(0.7033, device='cuda:0')

In [9]:
writer = SummaryWriter(logdir='results/runs')
input_ = (question, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task_tokens)
writer.add_graph(model, input_to_model=input_)
writer.close()

/aloui/vilbert-multi-task/vilbert/vilbert.py:1333: TracerWarning: resize_ can't be represented in the JIT at the moment, so we won't connect any uses of this value with its current trace. If you happen to use it again, it will show up as a constant in the graph.
  mask_tokens = input_txt.new().resize_(input_txt.size(0), 1).fill_(1)
/aloui/vilbert-multi-task/vilbert/vilbert.py:1686: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if pooled_output.size(0) % 2 == 0:
/home/aloui/miniconda3/envs/vilbert-mt/lib/python3.6/site-packages/torch/tensor.py:461: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).


Tracer cannot infer type of (tensor([[-23.9768, -17.3129, -18.0523,  ..., -18.6565, -18.3060, -14.8244],
        [-26.2138, -18.8952, -17.7499,  ..., -17.4112, -15.3365, -17.8206],
        [-24.8113, -16.7020, -11.6527,  ..., -15.2050, -14.3076, -10.0045],
        ...,
        [-27.0051, -15.3371, -20.5083,  ..., -19.9755, -16.9174, -17.5259],
        [-21.7966, -14.3504, -19.0064,  ..., -13.6335, -11.8910, -26.7599],
        [-23.3009, -11.6584, -19.6952,  ..., -18.6055, -13.8711, -18.0167]],
       device='cuda:0', grad_fn=<AddmmBackward>), tensor([[-11.6474, -10.8408,  -8.1201,  ..., -13.7666, -13.3638, -14.8178],
        [-14.1130,  -8.2416, -11.8985,  ..., -15.3526, -14.5676, -16.1072],
        [-10.0281,  -8.5481, -12.1849,  ...,  -9.9922, -13.6915, -11.6269],
        ...,
        [-20.1335, -18.3920, -17.5234,  ..., -20.9741, -23.5247, -22.5615],
        [ -6.5798,  -8.0710,  -7.1975,  ...,  -9.0155,  -8.8422, -10.2870],
        [-18.6119, -14.4927, -17.6004,  ..., -18.2182, -18

RuntimeError: Tracer cannot infer type of (tensor([[-23.9768, -17.3129, -18.0523,  ..., -18.6565, -18.3060, -14.8244],
        [-26.2138, -18.8952, -17.7499,  ..., -17.4112, -15.3365, -17.8206],
        [-24.8113, -16.7020, -11.6527,  ..., -15.2050, -14.3076, -10.0045],
        ...,
        [-27.0051, -15.3371, -20.5083,  ..., -19.9755, -16.9174, -17.5259],
        [-21.7966, -14.3504, -19.0064,  ..., -13.6335, -11.8910, -26.7599],
        [-23.3009, -11.6584, -19.6952,  ..., -18.6055, -13.8711, -18.0167]],
       device='cuda:0', grad_fn=<AddmmBackward>), tensor([[-11.6474, -10.8408,  -8.1201,  ..., -13.7666, -13.3638, -14.8178],
        [-14.1130,  -8.2416, -11.8985,  ..., -15.3526, -14.5676, -16.1072],
        [-10.0281,  -8.5481, -12.1849,  ...,  -9.9922, -13.6915, -11.6269],
        ...,
        [-20.1335, -18.3920, -17.5234,  ..., -20.9741, -23.5247, -22.5615],
        [ -6.5798,  -8.0710,  -7.1975,  ...,  -9.0155,  -8.8422, -10.2870],
        [-18.6119, -14.4927, -17.6004,  ...,

In [10]:
model

VILBertForVLTasks(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (task_embeddings): Embedding(20, 768)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): FusedLayerNorm(torch.Size([1024]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=Tr

In [33]:
vil_prediction[0][2969]

tensor(10.9015, device='cuda:0', grad_fn=<SelectBackward>)

In [24]:
logits

tensor([2969, 2785, 2293,  878,   89, 2621, 2134, 2621, 1403,  425, 1027,  411,
         425, 1403,  444, 1076, 2218, 2621,  402, 2621, 1136,  425,  711, 2785,
        2681, 2681,  990,  425, 1684, 2594], device='cuda:0')

In [25]:
one_hots[0][2969]

tensor(1., device='cuda:0')

In [29]:
target.shape

torch.Size([30, 3129])